# Build a cohort tutorial

**Example use case:** 

<img src="../images/julia.png" alt="alt_text" align="left"
	width="150" height="150" />
Julia is an oncologist that specializes in female reproductive health. As part of her research, she is interested in  using existing data on uterine cancers. If possible, she would like to see multiple datatypes (gross imaging, genomic data, proteomic data, histology) that come from the same patient, so she can look for shared phenotypes to test for their potential as early diagnostics. Julia heard that the Cancer Data Aggregator has made it easy to search across multiple datasets created by NCI, and so has decided to start her search there.



Before Julia does any work, she needs to import several functions from cdapython:

- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

She also asks cdapython to report it's version so she can be sure she's using the one she means to.

In [1]:
from cdapython import Q, columns, unique_terms, query
import cdapython
import pandas as pd 
print(cdapython.__version__)
Q.set_host_url("http://35.192.60.10:8080/")

2022.6.9


<div style="background-color:#c1f5ed;color:black;padding:20px;">
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in four main endpoints:

<ul>
<li><b>subject:</b> A specific, unique individual: for e.g. a single human. When consent allows, a given entity will have a single subject ID that can be connected to all their studies and data across all datasets</li>
<li><b>researchsubject:</b> a person/plant/animal/microbe within a given study. An individual who participates in 3 studies will have 3 researchsubject IDs</li>
<li><b>specimen:</b> a tissue sample taken from a given subject, or a portion of the original sample. A given specimen will have only a single subject ID and a single research subject ID</li>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> Information about what medical diagnosis a researchsubject has</li>
<li><b>treatment:</b> Information about what medical treatment(s) were performed for a given diagnosis</li>
</ul>
Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.
</div>


<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
   
   Accordingly, to see what search fields are available, Julia starts by using the command `columns`:

In [2]:
columns().to_list()

['File.id',
 'File.identifier.system',
 'File.identifier.value',
 'File.label',
 'File.data_category',
 'File.data_type',
 'File.file_format',
 'File.associated_project',
 'File.drs_uri',
 'File.byte_size',
 'File.checksum',
 'File.data_modality',
 'File.imaging_modality',
 'File.dbgap_accession_number',
 'id',
 'identifier.system',
 'identifier.value',
 'species',
 'sex',
 'race',
 'ethnicity',
 'days_to_birth',
 'subject_associated_project',
 'vital_status',
 'age_at_death',
 'cause_of_death',
 'ResearchSubject.id',
 'ResearchSubject.identifier.system',
 'ResearchSubject.identifier.value',
 'ResearchSubject.member_of_research_project',
 'ResearchSubject.primary_diagnosis_condition',
 'ResearchSubject.primary_diagnosis_site',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.identifier.system',
 'ResearchSubject.Diagnosis.identifier.value',
 'ResearchSubject.Diagnosis.primary_diagnosis',
 'ResearchSubject.Diagnosis.age_at_diagnosis',
 'ResearchSubject.Diagnosis.morphology',

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
   
There are a lot of columns in the CDA data, but Julia is most interested in diagnosis data, so she filters the list to only those:

In [3]:
columns().to_list(filters="diagnosis")

['ResearchSubject.primary_diagnosis_condition',
 'ResearchSubject.primary_diagnosis_site',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.identifier.system',
 'ResearchSubject.Diagnosis.identifier.value',
 'ResearchSubject.Diagnosis.primary_diagnosis',
 'ResearchSubject.Diagnosis.age_at_diagnosis',
 'ResearchSubject.Diagnosis.morphology',
 'ResearchSubject.Diagnosis.stage',
 'ResearchSubject.Diagnosis.grade',
 'ResearchSubject.Diagnosis.method_of_diagnosis',
 'ResearchSubject.Diagnosis.Treatment.id',
 'ResearchSubject.Diagnosis.Treatment.identifier.system',
 'ResearchSubject.Diagnosis.Treatment.identifier.value',
 'ResearchSubject.Diagnosis.Treatment.treatment_type',
 'ResearchSubject.Diagnosis.Treatment.treatment_outcome',
 'ResearchSubject.Diagnosis.Treatment.days_to_treatment_start',
 'ResearchSubject.Diagnosis.Treatment.days_to_treatment_end',
 'ResearchSubject.Diagnosis.Treatment.therapeutic_agent',
 'ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site',
 'Re

<div style="background-color:#c1f5ed;color:black;padding:20px;">

To search the CDA, a user also needs to know what search terms are available. Each column will contain a huge amount of data, so retreiving all of the rows would be overwhelming. Instead, the CDA has a `unique_terms()` function that will return all of the unique values that populate the requested column. Like `columns`, `unique_terms` defaults to giving us an overview of the results, and can be filtered.
    
</div>

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Since Julia is interested specificially in uterine cancers, she uses the `unique_terms` function to see what data is available for 'ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site' and 'ResearchSubject.primary_diagnosis_site' to see if 'uterine' appears:

In [4]:
unique_terms("ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site").to_list()

['Brain',
 'Cervix',
 'Head - Face Or Neck, Nos',
 'Lymph Node(s) Paraaortic',
 'Other',
 'Pelvis',
 'Spine',
 'Unknown']

In [5]:
unique_terms("ResearchSubject.primary_diagnosis_site").to_list()

['Abdomen',
 'Abdomen, Mediastinum',
 'Adrenal Glands',
 'Adrenal gland',
 'Anus and anal canal',
 'Base of tongue',
 'Bile Duct',
 'Bladder',
 'Bones, joints and articular cartilage of limbs',
 'Bones, joints and articular cartilage of other and unspecified sites',
 'Brain',
 'Breast',
 'Bronchus and lung',
 'Cervix',
 'Cervix uteri',
 'Chest',
 'Chest-Abdomen-Pelvis, Leg, TSpine',
 'Colon',
 'Connective, subcutaneous and other soft tissues',
 'Corpus uteri',
 'Ear',
 'Esophagus',
 'Extremities',
 'Eye and adnexa',
 'Floor of mouth',
 'Gallbladder',
 'Gum',
 'Head',
 'Head and Neck',
 'Head-Neck',
 'Heart, mediastinum, and pleura',
 'Hematopoietic and reticuloendothelial systems',
 'Hypopharynx',
 'Intraocular',
 'Kidney',
 'Larynx',
 'Lip',
 'Liver',
 'Liver and intrahepatic bile ducts',
 'Lung',
 'Lung Phantom',
 'Lymph nodes',
 'Marrow, Blood',
 'Meninges',
 'Mesothelium',
 'Nasal cavity and middle ear',
 'Nasopharynx',
 'Not Reported',
 'Oropharynx',
 'Other and ill-defined digest

<div style="background-color:#c1f5ed;color:black;padding:20px;">
    
CDA makes multiple datasets searchable from a common interface, but does not harmonize the data. This means that researchers should review all the terms in a column, and not just choose the first one that fits, as there may be other similar terms available as well.
    
</div>

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Julia sees that "treatment_anatomic_site" does not have 'Uterine', but does have 'Cervix'. She also notes that both 'Uterus' and 'Uterus, NOS' are listed in the "primary_diagnosis_site" results. As she was initially looking for "uterine", Julia decides to expand her search a bit to account for variable naming schemes. So, she runs a fuzzy match filter on the "ResearchSubject.primary_diagnosis_site" for 'uter' as that should cover all variants:

In [6]:
unique_terms("ResearchSubject.primary_diagnosis_site").to_list(filters="uter")

['Cervix uteri', 'Corpus uteri', 'Uterus', 'Uterus, NOS']

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Just to be sure, Julia also searches for any other instances of "cervix":

In [7]:
unique_terms("ResearchSubject.primary_diagnosis_site").to_list(filters="cerv")

['Cervix', 'Cervix uteri']

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
With all her likely terms found, Julia begins to create a search that will get data for all of her terms. She does this by writing a series of `Q` statements that define what rows should be returned from each column. For the "treatment_anatomic_site", only one term is of interest, so she uses the `=` operator to get only exact matches:

In [8]:
Tsite = Q('ResearchSubject.Diagnosis.Treatment.treatment_anatomic_site = "Cervix"')

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
However, for "primary_diagnosis_site", Julia has several terms she wants to search with. Luckily, `Q` also can run fuzzy searches. It can also search more than one term at a time, so Julia writes one big `Q` statement to grab everything that is either 'uter' or 'cerv':

In [9]:
Dsite = Q('ResearchSubject.primary_diagnosis_site = "%uter%" OR ResearchSubject.primary_diagnosis_site = "%cerv%"')

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Finally, Julia adds her two queries together into one large one:

In [10]:
ALLDATA = Tsite.OR(Dsite)

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Now that Julia has a query, she can use it to look for data in any of the CDA endpoints. She starts by getting an overall summary of what data is available using `count`:

In [11]:
ALLDATA.count.run()

Getting results from database

Total execution time: 3452 ms


specimen_count : 40766

treatment_count : 3045

diagnosis_count : 3683

researchsubject_count : 4867

subject_count : 3740

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
It seems there's a lot of data that might work for Julias study! Since she is interested in the beginings of cancer, she decides to start by looking at  the researchsubject information, since that is where most of the diagnosis information is. She again gets a summary using `count`:

In [12]:
ALLDATA.researchsubject.count.run()

Getting results from database

Total execution time: 3613 ms


total : 4867

files : 324216

system,count
GDC,3589
PDC,104
IDC,1174
primary_diagnosis_condition,count
Squamous Cell Neoplasms,609
Adenomas and Adenocarcinomas,1671
"Cystic, Mucinous and Serous Neoplasms",487
Uterine Corpus Endometrial Carcinoma,104
Complex Epithelial Neoplasms,27
Complex Mixed and Stromal Neoplasms,320


<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Browsing the primary_diagnosis_condition data, Julia notices that there are a large number of research subjects that are Adenomas and Adenocarcinomas. Since Julia wants to look for common phenotypes in early cancers, she decides it might be easier to exclude the endocrine related data, as they might have different mechanisms. So she adds a new filter to her query:

In [13]:
Noadeno = Q('ResearchSubject.primary_diagnosis_condition != "Adenomas and Adenocarcinomas"')

NoAdenoData = ALLDATA.AND(Noadeno)

NoAdenoData.researchsubject.count.run()

Getting results from database

Total execution time: 3529 ms


total : 3196

files : 297923

system,count
GDC,1918
PDC,104
IDC,1174
primary_diagnosis_condition,count
"Cystic, Mucinous and Serous Neoplasms",487
Squamous Cell Neoplasms,609
Complex Mixed and Stromal Neoplasms,320
Uterine Corpus Endometrial Carcinoma,104
Myomatous Neoplasms,187
None,1175


<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
She then previews the actual metadata for researchsubject, subject, and file, to make sure that they have all the information she will need for her work:

In [14]:
NoAdenoData.researchsubject.run().to_dataframe()

Getting results from database

Total execution time: 3634 ms


,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,183e8d87-3476-46bc-a989-ed4b09809227,"[{'system': 'GDC', 'value': '183e8d87-3476-46b...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-C5-A8XH
1,310a3ea9-8c9e-4c52-8bf7-ab53933bb623,"[{'system': 'GDC', 'value': '310a3ea9-8c9e-4c5...",GENIE-MSK,Complex Mixed and Stromal Neoplasms,"Uterus, NOS",GENIE-MSK-P-0015737
2,4e0d653d-21f0-468d-91e6-bca55dcddfb0,"[{'system': 'GDC', 'value': '4e0d653d-21f0-468...",CGCI-HTMCP-CC,Squamous Cell Neoplasms,Cervix uteri,HTMCP-03-06-02239
3,6b74c07a-9932-4a80-8296-9db08d0c6964,"[{'system': 'GDC', 'value': '6b74c07a-9932-4a8...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-EX-A1H5
4,6bcba66e-2c17-4134-afad-c8ee85d4e1b9,"[{'system': 'GDC', 'value': '6bcba66e-2c17-413...",CGCI-HTMCP-CC,Squamous Cell Neoplasms,Cervix uteri,HTMCP-03-06-02409
...,...,...,...,...,...,...
95,C3L-00963__cptac_ucec,"[{'system': 'IDC', 'value': 'C3L-00963'}]",cptac_ucec,None,Uterus,C3L-00963
96,C3N-00753__cptac_ucec,"[{'system': 'IDC', 'value': 'C3N-00753'}]",cptac_ucec,None,Uterus,C3N-00753
97,TCGA-AP-A1DO__tcga_ucec,"[{'system': 'IDC', 'value': 'TCGA-AP-A1DO'}]",tcga_ucec,None,Uterus,TCGA-AP-A1DO
98,TCGA-AX-A05W__tcga_ucec,"[{'system': 'IDC', 'value': 'TCGA-AX-A05W'}]",tcga_ucec,None,Uterus,TCGA-AX-A05W


---

<div style="background-color:#c1f5ed;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a research study, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subject’s privacy. An individual who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
  <li><b>id:</b> The unique identifier for this researchsubject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the researchsubject had there</li>
  <li><b>member_of_research_project:</b> The name of the study/project that the subject particpated in</li>
  <li><b>primary_diagnosis_condition:</b> The cancer, disease or other condition under study</li>
  <li><b>primary_diagnosis_site:</b> The primary_disease_site that qualifies the researchsubject for the research_project</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

In [15]:
NoAdenoData.subject.run().to_dataframe()

Getting results from database

Total execution time: 3470 ms


,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
0,AD1430,"[{'system': 'GDC', 'value': 'AD1430'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
1,AD3861,"[{'system': 'GDC', 'value': 'AD3861'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
2,C3L-05848,"[{'system': 'IDC', 'value': 'C3L-05848'}]",Homo sapiens,None,None,None,NaN,[cptac_ucec],None,NaN,None
3,C3N-00323,"[{'system': 'GDC', 'value': 'C3N-00323'}, {'sy...",Homo sapiens,female,not reported,not reported,-28627.0,"[CPTAC3-Discovery, CPTAC-3, cptac_ucec]",Alive,NaN,Not Reported
4,GENIE-DFCI-006487,"[{'system': 'GDC', 'value': 'GENIE-DFCI-006487'}]",Homo sapiens,female,black or african american,not hispanic or latino,-22645.0,[GENIE-DFCI],Not Reported,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
95,GENIE-VICC-575150,"[{'system': 'GDC', 'value': 'GENIE-VICC-575150'}]",Homo sapiens,female,white,not hispanic or latino,-27028.0,[GENIE-VICC],Not Reported,NaN,None
96,HTMCP-03-06-02222,"[{'system': 'GDC', 'value': 'HTMCP-03-06-02222'}]",Homo sapiens,female,black or african american,not hispanic or latino,NaN,[CGCI-HTMCP-CC],Alive,NaN,None
97,TCGA-AJ-A8CT,"[{'system': 'GDC', 'value': 'TCGA-AJ-A8CT'}, {...",Homo sapiens,female,black or african american,not reported,-21515.0,"[TCGA-UCEC, tcga_ucec]",Alive,NaN,None
98,TCGA-D1-A175,"[{'system': 'GDC', 'value': 'TCGA-D1-A175'}, {...",Homo sapiens,female,white,not hispanic or latino,-17802.0,"[TCGA-UCEC, tcga_ucec]",Alive,NaN,None


---

<div style="background-color:#c1f5ed;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A subject is a specific, unique individual: for e.g. a single human. When consent allows, a given entity will have a single subject ID that can be connected to all their studies and data across all datasets</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the subject had there</li>
  <li><b>species:</b> The species of the subject</li>
  <li><b>sex:</b> A reference to the biological sex of the donor organism. </li>
  <li><b>race:</b> The race of the subject</li>
  <li><b>ethnicity:</b> The ethnicity of the subject</li>
  <li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days</li>
  <li><b>subject_associated_project:</b> An embedded array of the names of projects (studies) the subject was part of</li>
  <li><b>vital_status:</b> Whether the subject is alive</li>
  <li><b>age_at_death:</b> The number of days after first enrollment that the subject died</li>
  <li><b>cause_of_death:</b> The cause of death, if known</li>
</ul>  

</div>
    
---

In [16]:
NoAdenoData.file.run().to_dataframe()

Getting results from database

Total execution time: 3742 ms


,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,researchsubject_specimen_id,researchsubject_id,subject_id
0,07734405-fc4b-4fa8-b6f9-ffc94124d330,"[{'system': 'GDC', 'value': '07734405-fc4b-4fa...",979d6ef9-860d-4a0e-9e42-8eb2f2ddd34c.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,BEDPE,CGCI-HTMCP-CC,drs://dg.4DFC:07734405-fc4b-4fa8-b6f9-ffc94124...,80149,51a07c367fac962816220c6ba724c948,Genomic,None,phs000528,1673c9c4-0adc-4b78-b1c0-4a0d7fee8e6f,f07f2302-2276-4d5c-b52b-0b900464d39c,HTMCP-03-06-02340
1,415706a1-c58d-485c-aa3d-244a6760ed18,"[{'system': 'GDC', 'value': '415706a1-c58d-485...",cfd92d9c-f6ed-4a01-89a4-83acfa32b794.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:415706a1-c58d-485c-aa3d-244a6760...,43688,6bd747ef331eed0a5c2b59892435865a,Genomic,None,phs000528,2c7cdfb5-07a9-403a-892e-1100dc1e595f,4b477751-4768-4d87-bd18-cd9a801acae3,HTMCP-03-06-02417
2,513f94b7-ef01-4405-8b2d-7cd814403761,"[{'system': 'GDC', 'value': '513f94b7-ef01-440...",b22b6c29-57ad-4497-ac67-bdc545d02afc.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:513f94b7-ef01-4405-8b2d-7cd81440...,121533,95e1d8e6dada42bfb2312894c15e019e,Genomic,None,phs000528,b3467f65-dc04-44a3-a5ac-fa5dad498431,d86bf73c-b964-4fb6-8e49-c487ce2251a7,HTMCP-03-06-02068
3,5e65e42f-c1be-4f1e-a8aa-d7d8798aee26,"[{'system': 'GDC', 'value': '5e65e42f-c1be-4f1...",66c6778c-0553-483c-a395-db3aad8fe31a.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:5e65e42f-c1be-4f1e-a8aa-d7d8798a...,54335,9e1badc4c154ef61b9ce658640b7ad23,Genomic,None,phs000528,fb2563c5-650f-4cd1-9f73-b7d89ad79a45,d02a8fb3-0f3e-475f-b2f7-27732908a284,HTMCP-03-06-02217
4,707c1b5d-d343-48e0-afae-ee4913a03f7d,"[{'system': 'GDC', 'value': '707c1b5d-d343-48e...",46c46275-6659-44fb-814b-9738601495df.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,BEDPE,CGCI-HTMCP-CC,drs://dg.4DFC:707c1b5d-d343-48e0-afae-ee4913a0...,205680,484700cf4e5e687cdf3a4583ae948fe7,Genomic,None,phs000528,a6e3ab1e-171e-4d7d-9d19-a6952bff7dcd,89c3d38f-8eda-4f62-bb9d-8d3b818c969f,HTMCP-03-06-02427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,513f94b7-ef01-4405-8b2d-7cd814403761,"[{'system': 'GDC', 'value': '513f94b7-ef01-440...",b22b6c29-57ad-4497-ac67-bdc545d02afc.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:513f94b7-ef01-4405-8b2d-7cd81440...,121533,95e1d8e6dada42bfb2312894c15e019e,Genomic,None,phs000528,0baf0fe2-e569-4d68-ad4a-507c5d813c4c,d86bf73c-b964-4fb6-8e49-c487ce2251a7,HTMCP-03-06-02068
96,542a5563-4db3-42f5-bd07-97e94490d02f,"[{'system': 'GDC', 'value': '542a5563-4db3-42f...",4410dfdd-a824-49c8-945f-6d4e2165fbab.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,VCF,CGCI-HTMCP-CC,drs://dg.4DFC:542a5563-4db3-42f5-bd07-97e94490...,30249,3134d728acb4deb20681beb25a8efbe7,Genomic,None,phs000528,80044043-11ff-4fe6-a9eb-2a8f20efdb15,18bce2c3-5060-4282-b7a0-11ada9a3eebf,HTMCP-03-06-02333
97,62374de1-b93f-4fa5-b248-d34bd09f82d8,"[{'system': 'GDC', 'value': '62374de1-b93f-4fa...",2cb4c39b-e508-47a6-8b58-3f6bf2033927.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,BEDPE,CGCI-HTMCP-CC,drs://dg.4DFC:62374de1-b93f-4fa5-b248-d34bd09f...,109310,29671513576f3bcfd10ae24021294bca,Genomic,None,phs000528,3e4e9e49-35af-4fbf-a386-ed9908f98144,cef5dedc-fd37-4653-b4b8-6c9d325c990d,HTMCP-03-06-02054
98,74e27c67-8e5f-4f63-b143-250424456bba,"[{'system': 'GDC', 'value': '74e27c67-8e5f-4f6...",58fd73fa-8129-4777-b424-58d553d8f3da.wgs.BRASS...,Somatic Structural Variation,Structural Rearrangement,BEDPE,CGCI-HTMCP-CC,drs://dg.4DFC:74e27c67-8e5f-4f63-b143-25042445...,83204,41dc96b20a61d2d7c89323c3b5a3e99d,Genomic,None,phs000528,48a6ea79-e75e-429b-ba14-352019e6d120,a78


---

<div style="background-color:#c1f5ed;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this file</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the file had there</li>
  <li><b>label:</b> The full name of the file</li>
  <li><b>data_catagory:</b> A desecription of the kind of general kind data the file holds</li>
  <li><b>data_type:</b> A more specific descripton of the data type</li>
  <li><b>file_format:</b> String to identify the full file extension including compression extensions</li>
  <li><b>associated_project:</b> The name the data center uses for the study this file was generated for</li>
  <li><b>drs_uri:</b> A unique identifier that can be used to retreive this specific file from a server</li>
  <li><b>byte_size:</b> Size of the file in bytes</li>
  <li><b>checksum:</b> The md5 value for the file</li>
  <li><b>data_modality:</b> Describes the biological nature of the information gathered as the result of an activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging"</li>
  <li><b>imaging_modality:</b> For files with the `data_modality` of "Imaging", a descriptor for the image type</li>
  <li><b>dbgap_accession_number:</b> The project id number for this data on dbGaP</li>
</ul>  

</div>
    
---


<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Finally, Julia wants to save these results to use for the future. Since the preview dataframes only show the first 100 results of each search, she uses the `paginator` function to get all the data from the subject and researchsubject endpoints into their own dataframes:

In [17]:
researchsubs = NoAdenoData.researchsubject.run()
rsdf = pd.DataFrame()
for i in researchsubs.paginator(to_df=True):
    rsdf = pd.concat([rsdf, i])

Getting results from database

Total execution time: 3481 ms


In [18]:
subs = NoAdenoData.subject.run()
subsdf = pd.DataFrame()
for i in subs.paginator(to_df=True):
    subsdf = pd.concat([subsdf, i])

Getting results from database

Total execution time: 3367 ms


In [19]:
rsdf

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,183e8d87-3476-46bc-a989-ed4b09809227,"[{'system': 'GDC', 'value': '183e8d87-3476-46b...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-C5-A8XH
1,310a3ea9-8c9e-4c52-8bf7-ab53933bb623,"[{'system': 'GDC', 'value': '310a3ea9-8c9e-4c5...",GENIE-MSK,Complex Mixed and Stromal Neoplasms,"Uterus, NOS",GENIE-MSK-P-0015737
2,4e0d653d-21f0-468d-91e6-bca55dcddfb0,"[{'system': 'GDC', 'value': '4e0d653d-21f0-468...",CGCI-HTMCP-CC,Squamous Cell Neoplasms,Cervix uteri,HTMCP-03-06-02239
3,6b74c07a-9932-4a80-8296-9db08d0c6964,"[{'system': 'GDC', 'value': '6b74c07a-9932-4a8...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,TCGA-EX-A1H5
4,6bcba66e-2c17-4134-afad-c8ee85d4e1b9,"[{'system': 'GDC', 'value': '6bcba66e-2c17-413...",CGCI-HTMCP-CC,Squamous Cell Neoplasms,Cervix uteri,HTMCP-03-06-02409
...,...,...,...,...,...,...
91,db6db2c4-210b-4f7a-a80c-2603957b7bc5,"[{'system': 'GDC', 'value': 'db6db2c4-210b-4f7...",GENIE-DFCI,Squamous Cell Neoplasms,Cervix uteri,GENIE-DFCI-028657
92,e87136c1-29a2-4e4c-aae8-05ce8b4bb090,"[{'system': 'GDC', 'value': 'e87136c1-29a2-4e4...",GENIE-DFCI,Complex Epithelial Neoplasms,Cervix uteri,GENIE-DFCI-008895
93,e9fa66f5-282c-416a-9dcf-46714ea12fcf,"[{'system': 'GDC', 'value': 'e9fa66f5-282c-416...",TCGA-UCEC,"Cystic, Mucinous and Serous Neoplasms",Corpus uteri,TCGA-DI-A2QU
94,f93ccd2d-3c77-4bd3-8cac-5203bd57735f,"[{'system': 'GDC', 'value': 'f93ccd2d-3c77-4bd...",TCGA-CESC,"Cystic, Mucinous and Serous Neoplasms",Cervix uteri,TCGA-Q1-A5R1


In [20]:
subsdf

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
0,AD1430,"[{'system': 'GDC', 'value': 'AD1430'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
1,AD3861,"[{'system': 'GDC', 'value': 'AD3861'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
2,C3L-05848,"[{'system': 'IDC', 'value': 'C3L-05848'}]",Homo sapiens,None,None,None,NaN,[cptac_ucec],None,NaN,None
3,C3N-00323,"[{'system': 'GDC', 'value': 'C3N-00323'}, {'sy...",Homo sapiens,female,not reported,not reported,-28627.0,"[CPTAC3-Discovery, CPTAC-3, cptac_ucec]",Alive,NaN,Not Reported
4,GENIE-DFCI-006487,"[{'system': 'GDC', 'value': 'GENIE-DFCI-006487'}]",Homo sapiens,female,black or african american,not hispanic or latino,-22645.0,[GENIE-DFCI],Not Reported,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
3,TCGA-EA-A410,"[{'system': 'GDC', 'value': 'TCGA-EA-A410'}, {...",Homo sapiens,female,white,not hispanic or latino,-18720.0,"[TCGA-CESC, tcga_cesc]",Alive,NaN,None
4,TCGA-EO-A2CH,"[{'system': 'GDC', 'value': 'TCGA-EO-A2CH'}, {...",Homo sapiens,female,not reported,not reported,-26966.0,"[TCGA-UCEC, tcga_ucec]",Dead,590.0,None
5,TCGA-FL-A3WE,"[{'system': 'GDC', 'value': 'TCGA-FL-A3WE'}, {...",Homo sapiens,None,None,None,NaN,"[TCGA-UCEC, tcga_ucec]",None,NaN,None
6,TCGA-N7-A59B,"[{'system': 'GDC', 'value': 'TCGA-N7-A59B'}, {...",Homo sapiens,female,white,not hispanic or latino,-25360.0,"[tcga_ucs, TCGA-UCS]",Alive,NaN,None


<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Then Julia uses the `id` fields in each result to join them together into one big dataset:

In [21]:
allmetadata = rsdf.set_index("subject_id").join(subsdf.set_index("id"), lsuffix='resub', rsuffix="subject")


In [22]:
allmetadata

,id,identifierresub,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,identifiersubject,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
AD100,0f08e2e9-9983-4204-972f-a630b7ab2c25,"[{'system': 'GDC', 'value': '0f08e2e9-9983-420...",FM-AD,Squamous Cell Neoplasms,Cervix uteri,"[{'system': 'GDC', 'value': 'AD100'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
AD1026,6d9d6cb9-652f-4749-b4c5-aa9e6b80de69,"[{'system': 'GDC', 'value': '6d9d6cb9-652f-474...",FM-AD,Complex Mixed and Stromal Neoplasms,"Uterus, NOS","[{'system': 'GDC', 'value': 'AD1026'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
AD10328,514fc104-1ee5-4701-8f45-9a011143f1e2,"[{'system': 'GDC', 'value': '514fc104-1ee5-470...",FM-AD,Squamous Cell Neoplasms,Cervix uteri,"[{'system': 'GDC', 'value': 'AD10328'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
AD10460,8c36611d-be2f-432a-afde-e684ab4333ea,"[{'system': 'GDC', 'value': '8c36611d-be2f-432...",FM-AD,"Cystic, Mucinous and Serous Neoplasms","Uterus, NOS","[{'system': 'GDC', 'value': 'AD10460'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
AD10485,0ad0fdda-dd96-48df-8edd-e5e471e9f680,"[{'system': 'GDC', 'value': '0ad0fdda-dd96-48d...",FM-AD,"Cystic, Mucinous and Serous Neoplasms","Uterus, NOS","[{'system': 'GDC', 'value': 'AD10485'}]",Homo sapiens,female,not reported,not reported,NaN,[FM-AD],Not Reported,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZJ-AB0H,TCGA-ZJ-AB0H__tcga_cesc,"[{'system': 'IDC', 'value': 'TCGA-ZJ-AB0H'}]",tcga_cesc,None,Cervix,"[{'system': 'GDC', 'value': 'TCGA-ZJ-AB0H'}, {...",Homo sapiens,female,not reported,not reported,-17869.0,"[TCGA-CESC, tcga_cesc]",Alive,NaN,None
TCGA-ZJ-AB0I,a4f13656-a941-498a-9ac9-f020ed559b35,"[{'system': 'GDC', 'value': 'a4f13656-a941-498...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,"[{'system': 'GDC', 'value': 'TCGA-ZJ-AB0I'}, {...",Homo sapiens,female,white,not hispanic or latino,-9486.0,"[TCGA-CESC, tcga_cesc]",Alive,NaN,None
TCGA-ZJ-AB0I,TCGA-ZJ-AB0I__tcga_cesc,"[{'system': 'IDC', 'value': 'TCGA-ZJ-AB0I'}]",tcga_cesc,None,Cervix,"[{'system': 'GDC', 'value': 'TCGA-ZJ-AB0I'}, {...",Homo sapiens,female,white,not hispanic or latino,-9486.0,"[TCGA-CESC, tcga_cesc]",Alive,NaN,None
TCGA-ZX-AA5X,4756acc0-4e96-44d4-b359-04d64dc7eb84,"[{'system': 'GDC', 'value': '4756acc0-4e96-44d...",TCGA-CESC,Squamous Cell Neoplasms,Cervix uteri,"[{'system': 'GDC', 'value': 'TCGA-ZX-AA5X'}, {...",Homo sapiens,female,white,not hispanic or latino,-23440.0,"[TCGA-CESC, tcga_cesc]",Alive,NaN,None


<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
And saves it out to a csv so she can browse it with Excel:

In [23]:
allmetadata.to_csv("allmetadata.csv")

<img src="../images/julia.png" align="left"
	width="50" height="50" />
   
Julia knows from her subject count summary that there are 33480 files associated with her subjects, which is likely far more than she needs. In the future, she would like to be able to run something like `file.count.run()` to see a summary of file types so she can filter these down based on data type. Then she could make one data set with histological data, another with MRIs, etc.

She would then take her smaller file list and import the files to somewhere that fits her analysis needs like Terra or the Seven Bridges CGC platform.

In [24]:
#files = NoAdenoData.file.run()
#filedf = pd.DataFrame()
#for i in files.paginator(to_df=True):
#    filedf = pd.concat([subsdf, i])